In [ ]:
# import libraries
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
import re

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

train_df = pd.read_csv("train-data.tsv", sep="\t", header = None)
test_df = pd.read_csv("valid-data.tsv", sep="\t", header = None)

In [ ]:
#Assigning column titles
train_df.columns = ['status', 'message']
test_df.columns = ['status', 'message']

print(train_df.head())

In [ ]:
#PREPROCESSING
#Normalise numbers

def replace_numbers(text):
    return re.sub(r"\d+", "NUM", text)

train_df['message'] = train_df['message'].apply(replace_numbers)
test_df['message'] = test_df['message'].apply(replace_numbers)

#Encoding Messages

messages = train_df['message'].astype(str).to_list()

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(messages)

word_index = tokenizer.word_index
print("Size of vocabulary:", len(word_index))

train_sequences = tokenizer.texts_to_sequences(train_df['message'])
test_sequences = tokenizer.texts_to_sequences(test_df['message'])

train_sequences = sequence.pad_sequences(train_sequences, maxlen=120)
test_sequences = sequence.pad_sequences(test_sequences, maxlen=120)

assert len(train_sequences) == len(train_df['status'])

#Encoding Status
train_df['status'] = train_df['status'].map({'ham': 0, 'spam': 1}).fillna(0).astype('int32').to_numpy()
test_df['status'] = test_df['status'].map({'ham': 0, 'spam': 1}).fillna(0).astype('int32').to_numpy()

In [ ]:
#Model
vocab_cap = min(10000, len(word_index) + 1)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = vocab_cap, output_dim = 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_sequences, train_df['status'], epochs=10)

In [ ]:
results = model.evaluate(test_sequences, test_df['status'])
print('Test loss, acc:', results)

In [ ]:
#Converting encodings to words

inverse_code = {value: key for (key, value) in word_index.items()}

def reverse(num_list):
  text = ''
  for num in num_list:
    if num != 0:
      text += inverse_code[num] + ' '
  text = text[:-1] + '.'
  return text

print(reverse(train_sequences[0]))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text = replace_numbers(pred_text)
  pred_seq = tokenizer.texts_to_sequences([pred_text])
  pred_seq = sequence.pad_sequences(pred_seq, maxlen=120)

  prediction = model.predict(pred_seq)[0]
  if prediction < 0.5:
    prediction = [prediction, 'ham']
  else:
    prediction = [prediction, 'spam']

  return prediction

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)

    print(prediction, ans)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
